In [1]:
from schema_export import ViewSchemaExporter,TableSchemaExporter
from df_creator import DFCreator
import pandas as pd

In [1]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import *

In [3]:
schema = 'SAPHANADB'
source = 'VBAP'
source_type = 'TABLE'

In [4]:
sap_con = '{"host" : "40.87.84.72","port" : 30215,"user" : "system","password" : "Metro#123"}'
sql_con = 'Server=tcp:yashtesting.database.windows.net;Database=test;Uid=yash@yashtesting;Pwd=Myageis@20;Encrypt=yes;Connection Timeout=30;'
sql_odbc_con = 'Driver={ODBC Driver 17 for SQL Server};'+sql_con

In [5]:
if source_type == 'VIEW':
    se = ViewSchemaExporter(sap_con_string=sap_con,sql_con_string=sql_odbc_con,sap_view= f'{schema}.{source}')
    se.replicate_sap_view()
elif source_type == 'TABLE':
    se = TableSchemaExporter(sap_con_string=sap_con,sql_con_string=sql_odbc_con,sap_table= f'{schema}.{source}')
    se.replicate_sap_table()

Connecting to DB


In [3]:
subscription_id = '938ace66-9598-4029-b6bb-429929b03761'
rg_name = 'celebal_rnd'
df_name = 'celebaladf'
client_id = 'b628371b-654f-4848-b214-c8553f2fc665' 
secret = '/JCA4now2LAn1/L4aa+ICfmTumPRryW.'
tenant = 'e4e34038-ea1f-4882-b6e8-ccd776459ca0'
rg_params = {'location':'eastus'}
df_params = {'location':'eastus'}

In [7]:
credentials = ServicePrincipalCredentials(client_id=client_id, secret=secret, tenant=tenant)
resource_client = ResourceManagementClient(credentials, subscription_id)
adf_client = DataFactoryManagementClient(credentials, subscription_id)

# Linked Services

## SAP Linked Service

In [8]:
sap_ls_name = 'sapHanaLinkedService'
ir_reference = IntegrationRuntimeReference(reference_name='integrationRuntime3')
sap_pass = SecureString(value='Metro#123')
ls_sap_hana = SapHanaLinkedService(server='40.87.84.72:30215',user_name='system',password=sap_pass,authentication_type='Basic',connect_via=ir_reference)
ls = adf_client.linked_services.create_or_update(rg_name, df_name, sap_ls_name, ls_sap_hana)

## SQL Linked Service

In [9]:
sql_ls_name = 'sqlLinkedServiceYash'
sql_string = SecureString(value=sql_con)
ls_azure_sql = AzureSqlDatabaseLinkedService(connection_string=sql_string)
sql_ls = adf_client.linked_services.create_or_update(rg_name, df_name, sql_ls_name, ls_azure_sql)

# Datasets

## SAP Dataset

In [10]:
ds_name = 'SAP_Dataset'
ds_ls = LinkedServiceReference(reference_name=sap_ls_name)
ds_sap_table = RelationalTableDataset(linked_service_name=ds_ls)
ds = adf_client.datasets.create_or_update(rg_name, df_name, ds_name, ds_sap_table)

## SQL Dataset

In [11]:
dsOut_name = 'SQL_Dataset'
dsOut_def = {
    "linkedServiceName": {
        "referenceName": f"{sql_ls_name}",
        "type": "LinkedServiceReference"
    },
    "parameters": {
        "TABLENAME": {
            "type": "string"
        }
    },
    "type": "AzureSqlTable",
    "schema": [],
    "typeProperties": {
        "tableName": {
            "value": "@dataset().TABLENAME",
            "type": "Expression"
        }
    }
}
dsOut_azure_sql = AzureSqlTableDataset.from_dict(dsOut_def)
dsOut = adf_client.datasets.create_or_update(rg_name, df_name, dsOut_name, dsOut_azure_sql)

In [12]:
type(dsOut_azure_sql)

azure.mgmt.datafactory.models.azure_sql_table_dataset.AzureSqlTableDataset

## Source and Sink

In [13]:
mapping = se.mapping

In [14]:
def make_source_query() -> str:
    source_name = f'"{schema}"."{source}"'
    return f'SELECT {",".join(mapping.keys())} from {source_name}'

def make_precopy_script(sql_table:str) -> str:
    return f'TRUNCATE TABLE [dbo].[{source}]'

In [15]:
source_type = RelationalSource(query=make_source_query())
destination_type = SqlSink()

In [16]:
dsin_ref = DatasetReference(ds_name)
dsOut_ref = DatasetReference(dsOut_name,parameters={
    "TABLENAME":f"[dbo].[{source}]"
})

In [17]:
def make_translator()->TabularTranslator:
    source = mapping     
    mappings = []
    for key in source:
        obj = {}
        obj['source'] = {"name":key}
        obj['sink'] = {"name":source[key]}
        mappings.append(obj)
    return TabularTranslator.from_dict({"type":"TabularTranslator","mappings":mappings})

In [18]:
trans = make_translator()

In [19]:
copy_activity = CopyActivity('S4Hana to SQL Copy',inputs=[dsin_ref], outputs=[dsOut_ref],source=source_type, sink=destination_type,translator=trans)
p_name = 'latestPyCode'
p_obj = PipelineResource(activities=[copy_activity])
p = adf_client.pipelines.create_or_update(rg_name, df_name, p_name, p_obj)